# __Search image by image__

**Understanding image digitization techniques**
---
👉 Images are high-dimensional data (height x width x channel (RGB) x color intensity), which means nothing if the information of each pixel is randomly generated. In other words, each pixel can only be recognized as an image if it has a specific pattern associated with the surrounding pixel. This means that an image can be represented on a lower-dimensional characteristic vector than it really is. Recently, studies using artificial intelligence to numericalize and express each image in a low-dimensional space according to the similarity of the meaning of each image have been conducted in various ways, and these have been used in various names such as image digitization, vectorization, and embedding.

**In this tutorial**
---
👉 This tutorial will use the MNIST handwriting dataset. Each image is a fixed size (28x28 = 784 pixels) with a value between 0-1 and provides a number from 0-9 written by many people with the correct answer. It consists of 1,000 learning datasets and 200 testing datasets.

Let's create a model that uses ThanoSQL to enter handwriting data and search for images similar to the input image within the DB.

## __0. Check dataset__

To use the query syntax of ThanoSQL, you must create an API token and run the query below, as mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/quick_start/how_to_use_ThanoSQL/#5-thanosql).

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

In [5]:
%%thanosql
COPY mnist_train 
OPTIONS(overwrite = True)
FROM "tutorial_data/mnist_data/mnist_train.csv"

Time Taken: 0.51 seconds
Success


In [7]:
%%thanosql
COPY mnist_test 
OPTIONS(overwrite = True)
FROM "tutorial_data/mnist_data/mnist_test.csv"

Time Taken: 0.45 seconds
Success


__OPTIONS__ : 

When __overwrite is true__, the user can create a data table with the same name as the previously created data table.  
On the other hand, when __overwrite is False__, the user cannot create a data table with the same name as the previously created data table.

## __1. Check dataset__

Use the mnist_train table stored in the ThanoSQL DB to create a handwriting classification model. The mnist_train table is a table containing the path, file name, and label information where MNIST image files are stored. Run the query statement below and check the contents of the table.

In [8]:
%%thanosql
SELECT * 
FROM mnist_train 
LIMIT 5

Time Taken: 0.44 seconds


,image_path,filename,label
0,tutorial_data/mnist_data/train/6782.jpg,6782.jpg,5
1,tutorial_data/mnist_data/train/1810.jpg,1810.jpg,5
2,tutorial_data/mnist_data/train/33617.jpg,33617.jpg,5
3,tutorial_data/mnist_data/train/27802.jpg,27802.jpg,5
4,tutorial_data/mnist_data/train/50677.jpg,50677.jpg,5


The mnist_train table contains the following information: The "6782.jpg" image file is a handwritten image with the number 5.

- img_path: Image Path
- filename: file name
- label : image label

## __2. Creating an Image Numerical Model__

Create an image quantification model using the mnist_train table from the previous step. Execute the query syntax below to create a model named my_image_search_model.

(Estimated time required for query execution: 1 min)

In [30]:
%%thanosql
BUILD MODEL my_image_search_model
USING SimCLR
OPTIONS (
    image_col="img_path",
    max_epochs=5,
    overwrite=True
    )
AS 
SELECT * 
FROM mnist_train

Time Taken: 332.16 seconds
Success


# __Query Details__ 
- Create and train a model called my_image_search_model using the query syntax "__BUILD MODEL__".
- The "__USING__" query syntax specifies the use of the SimCLR model as the base model.
- Specify the options to use for model creation via the query syntax "__OPTIONS__".  
    -  "image_col" : Column containing image path in data table (Default: "image_path")    
    -  "max_epochs" : Number of dataset learnings to generate image quantization models

Use the query syntax below to view the results of the image quantification. Embedding images `mnist_test` using the query syntax '__CONVERTUSING__' `my_image_search_model`


In [37]:
%%thanosql
CONVERT USING my_image_search_model
OPTIONS (
    table_name= "mnist_test",
    image_col="img_path"
    )
AS 
SELECT * 
FROM mnist_test

Time Taken: 2.19 seconds


,image_path,filename,label,my_image_search_model_simclr
0,tutorial_data/mnist_data/test/5099.jpg,5099.jpg,6,"[0.8018283247947693, 0.79114830493927, 1.35917..."
1,tutorial_data/mnist_data/test/9239.jpg,9239.jpg,6,"[0.5109610557556152, 0.7103637456893921, 0.762..."
2,tutorial_data/mnist_data/test/2242.jpg,2242.jpg,6,"[0.5548616051673889, 0.5135034322738647, 0.882..."
3,tutorial_data/mnist_data/test/3451.jpg,3451.jpg,6,"[0.6456339955329895, 0.8264403939247131, 0.982..."
4,tutorial_data/mnist_data/test/2631.jpg,2631.jpg,6,"[0.9043697118759155, 1.001702070236206, 0.8401..."
...,...,...,...,...
195,tutorial_data/mnist_data/test/8045.jpg,8045.jpg,8,"[0.900598406791687, 0.8024667501449585, 0.9821..."
196,tutorial_data/mnist_data/test/9591.jpg,9591.jpg,8,"[0.7340981960296631, 0.24722115695476532, 1.25..."
197,tutorial_data/mnist_data/test/7425.jpg,7425.jpg,8,"[0.6172817349433899, 0.278918594121933, 0.4667..."
198,tutorial_data/mnist_data/test/2150.jpg,2150.jpg,8,"[1.0326189994812012, 0.8284595012664795, 0.923..."


__Query Details__ 
- The query syntax "__CONVERT USING__" uses `my_image_search_model` as an algorithm for image quantification.   
- Define the variables required for image quantification with the query syntax "__OPTIONS__". 
    - "table_name" : Defines the table name to be stored in the ThanoSQL DB.
    - "image_col" : Defines the column name containing the image file path in the data table. (DEFAULT: "image_path")

Create a new column named `my_image_search_model_simclr` in the table `mnist_test` and save the quantization results.

## __3. Search for similar images using image quantization models__

This step uses the my_image_search_model image quantization model and the test table to search for images similar to the "923.jpg" image file (handwriting 8).

In [41]:
%%thanosql
SEARCH IMAGE images='tutorial_data/mnist_data/test/923.jpg' 
USING my_image_search_model 
AS
SELECT * 
FROM mnist_test

Time Taken: 6.62 seconds


,image_path,my_image_search_model_simclr,my_image_search_model_simclr_similarity1
0,tutorial_data/mnist_data/test/2778.jpg,"[0.4345078468322754, 0.4801121950149536, 0.849...",0.904738
1,tutorial_data/mnist_data/test/616.jpg,"[0.8675670623779297, 1.101262092590332, 0.7946...",0.940482
2,tutorial_data/mnist_data/test/1809.jpg,"[0.911918044090271, 0.5684484839439392, 0.7830...",0.929987
3,tutorial_data/mnist_data/test/5165.jpg,"[1.4594722986221313, 1.5304396152496338, 0.906...",0.946843
4,tutorial_data/mnist_data/test/8600.jpg,"[0.7580893635749817, 0.579352617263794, 0.8556...",0.901757
...,...,...,...
195,tutorial_data/mnist_data/test/4949.jpg,"[0.8616079688072205, 1.2864174842834473, 0.899...",0.929408
196,tutorial_data/mnist_data/test/6146.jpg,"[0.5501160621643066, 0.8333878517150879, 1.228...",0.933200
197,tutorial_data/mnist_data/test/2002.jpg,"[1.0146446228027344, 0.6418188810348511, 0.629...",0.933350
198,tutorial_data/mnist_data/test/4255.jpg,"[0.8223161697387695, 0.5770737528800964, 1.053...",0.936178


__Query Details__  

- The query syntax "___SEARCH IMAGE images=__" defines the image file you want to search for.  <br>
- "__USING__" defines the model used for image quantification.<br>
- The "__AS__" query syntax defines the embedding table to use for searches. Use table `mnist_embds`

Run the following query to output the "__SEARCH__" result using the "__PRINT__" query syntax in ThanoSQL to output the top four most similar. We've only done a little bit of learning, but you can see that it's outputting an image similar to 8.

In [42]:
%%thanosql
PRINT IMAGE 
AS (
    SELECT image_path, my_image_search_model_simclr_similarity1 
    FROM (
        SEARCH IMAGE images='tutorial_data/mnist_data/test/923.jpg' 
        USING my_image_search_model 
        AS 
        SELECT * 
        FROM mnist_test
        )
    ORDER BY my_image_search_model_simclr_similarity1 DESC 
    LIMIT 4
    )

Time Taken: 1.12 seconds
/home/jovyan/tutorial_data/mnist_data/test/923.jpg


/home/jovyan/tutorial_data/mnist_data/test/685.jpg


/home/jovyan/tutorial_data/mnist_data/test/6573.jpg


/home/jovyan/tutorial_data/mnist_data/test/5087.jpg


__Notes for reference__ 

The basic learning options of the image similarity search algorithm are learned to recognize the image as the same regardless of the image's left-right inversion, color change, etc. This is because a dog's picture should be recognized as a dog even if it is flipped or changed in color. If color changes are important, such as clothing images, or if vertical and horizontal twists are important, such as numbers, the options should be changed when learning. This tutorial shows the characteristics of these image similarity searches.